In [17]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



The nvcc4jupyter extension is already loaded. To reload it, use:Collecting git+https://github.com/andreinechaev/nvcc4jupyter.git
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\a1742\appdata\local\temp\pip-req-build-3mbbzg0h
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'

  %reload_ext nvcc4jupyter


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\A1742\AppData\Local\Temp\pip-req-build-3mbbzg0h'

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\A1742\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)



In [20]:
%%cuda_group_save -g "knn" -n "main.cu"

// Required header files
// 必要的头文件包含
#include <iostream>     // Standard I/O operations / 标准输入输出
#include <fstream>      // File operations / 文件操作
#include <vector>       // Vector container / 向量容器
#include <string>       // String processing / 字符串处理
#include <cstring>      // C-style string operations / C风格字符串操作
#include <algorithm>    // Standard algorithms / 算法库
#include <cuda_runtime.h> // CUDA runtime / CUDA运行时

// Constants definition / 常量定义
#define THREADS 32         // Number of threads per block / 每个block的线程数
#define IMAGESIZE 784      // Image size (28x28 = 784 pixels) / 图像大小 (28x28 = 784像素)

// Training sample structure definition / 定义训练样本结构体
struct TrainingSample {
    int label;                  // Image label (0-9) / 图像标签 (0-9)
    float image[IMAGESIZE];     // Image data array / 图像数据数组
};

// CUDA kernel: Single step of bitonic sort
// CUDA核函数：双调排序的单步操作
__global__ void bitonicSortKernel(float* d_distances, int* d_labels, int j, int k, int num_samples) {
    // Calculate global index of current thread
    // 计算当前线程的全局索引
    unsigned int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i >= num_samples) return;  // Boundary check / 边界检查
    
    // Calculate index of paired element for comparison
    // 计算要比较的配对元素的索引
    unsigned int ixj = i ^ j;
    if (ixj > i && ixj < num_samples) {
        if ((i & k) == 0) {  // Determine sort direction / 决定排序方向
            // Swap if necessary (both distances and labels)
            // 如果需要交换，则同时交换距离和标签
            if (d_distances[i] > d_distances[ixj]) {
                // Swap distances / 交换距离
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;
                
                // Swap corresponding labels / 交换对应的标签
                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        }
    }
}

// GPU device function: Complete bitonic sort
// GPU设备函数：执行完整的双调排序
__device__ void bitonicSort(float* d_distances, int* d_labels, int num_samples) {
    // Calculate smallest power of 2 greater than or equal to sample count
    // 计算大于等于样本数的最小2的幂
    int pow2_size = 1;
    while (pow2_size < num_samples) pow2_size *= 2;
    
    // Main loop of bitonic sort
    // 双调排序的主循环
    for (int k = 2; k <= pow2_size; k *= 2) {
        for (int j = k/2; j > 0; j /= 2) {
            int num_blocks = (num_samples + THREADS - 1) / THREADS;
            bitonicSortKernel<<<num_blocks, THREADS>>>(d_distances, d_labels, j, k, num_samples);
            cudaDeviceSynchronize();  // Ensure all threads complete / 确保所有线程完成
        }
    }
}

// CUDA kernel: Compute Euclidean distances
// CUDA核函数：计算欧几里得距离
__global__ void computeEuclideanDistances(float* d_images, float* d_testImage, 
                                        float* d_distances, int* d_labels,
                                        int* d_train_labels, int num_samples) {
    // Calculate global index of current thread
    // 计算当前线程的全局索引
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_samples) {
        float sum = 0.0f;
        // Calculate Euclidean distance between current training image and test image
        // 计算当前训练图像与测试图像的欧几里得距离
        for (int i = 0; i < IMAGESIZE; ++i) {
            float diff = d_images[idx * IMAGESIZE + i] - d_testImage[i];
            sum += diff * diff;
        }
        d_distances[idx] = sqrtf(sum);
        d_labels[idx] = d_train_labels[idx];  // Save corresponding label / 保存对应的标签
    }
}

// Load MNIST data from binary file
// 从二进制文件加载MNIST数据
bool loadData(const std::string& filename, std::vector<TrainingSample>& samples) {
    std::cout << "Attempting to open file: " << filename << std::endl;
    std::ifstream file(filename, std::ios::binary);
    if (!file) {
        std::cerr << "Unable to open file " << filename << std::endl;
        return false;
    }

    // Read number of samples / 读取样本数量
    int num_samples = 0;
    file.read(reinterpret_cast<char*>(&num_samples), sizeof(int));
    if (!file) {
        std::cerr << "Error reading number of samples." << std::endl;
        return false;
    }

    samples.resize(num_samples);

    // Read label and image data for each sample
    // 读取每个样本的标签和图像数据
    for (int i = 0; i < num_samples; ++i) {
        file.read(reinterpret_cast<char*>(&samples[i].label), sizeof(int));
        file.read(reinterpret_cast<char*>(samples[i].image), sizeof(float) * IMAGESIZE);
    }

    file.close();
    return true;
}

int main() {
    // Declare vectors for storing training and test samples
    // 声明存储训练和测试样本的向量
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    // Load training data / 加载训练数据
    if (!loadData("train_mnist.bin", train_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << train_samples.size() << " training samples." << std::endl;

    // Load testing data / 加载测试数据
    if (!loadData("test_mnist.bin", test_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << test_samples.size() << " testing samples." << std::endl;

    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    // Prepare host memory for training data
    // 为训练数据准备主机内存
    float* h_train_images = new float[num_trainsamples * IMAGESIZE];
    int* h_train_labels = new int[num_trainsamples];

    // Convert training data to continuous arrays
    // 将训练数据转换为连续数组
    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, sizeof(float) * IMAGESIZE);
    }

    // Allocate GPU memory / 分配GPU内存
    float* d_train_images;
    int* d_train_labels;
    cudaMalloc(&d_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    cudaMalloc(&d_train_labels, num_trainsamples * sizeof(int));

    // Copy training data to GPU / 将训练数据复制到GPU
    cudaMemcpy(d_train_images, h_train_images, num_trainsamples * IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_train_labels, h_train_labels, num_trainsamples * sizeof(int), cudaMemcpyHostToDevice);

    // KNN parameters / KNN参数设置
    int k = 10;  // Choose 10 nearest neighbors / 选择10个最近邻
    int correct_predictions = 0;

    // Process each test sample / 处理每个测试样本
    for (int t = 0; t < num_testsamples; ++t) {
        int test_label = test_samples[t].label;
        
        // Allocate GPU memory for test image and copy data
        // 为测试图像分配GPU内存并复制数据
        float* d_test_image;
        cudaMalloc(&d_test_image, IMAGESIZE * sizeof(float));
        cudaMemcpy(d_test_image, test_samples[t].image, IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);

        // Allocate GPU memory for distances and labels
        // 为距离和标签分配GPU内存
        float* d_distances;
        int* d_sort_labels;
        cudaMalloc(&d_distances, num_trainsamples * sizeof(float));
        cudaMalloc(&d_sort_labels, num_trainsamples * sizeof(int));

        // Configure CUDA kernel execution parameters
        // 配置CUDA kernel的运行参数
        int threadsPerBlock = 256;
        int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;
        
        // Compute Euclidean distances
        // 计算欧几里得距离
        computeEuclideanDistances<<<blocksPerGrid, threadsPerBlock>>>(
            d_train_images, d_test_image, d_distances, d_sort_labels, d_train_labels, num_trainsamples
        );

        // Sort distances / 对距离进行排序
        bitonicSort<<<1, 1>>>(d_distances, d_sort_labels, num_trainsamples);

        // Allocate host memory and copy k-nearest neighbors results
        // 分配主机内存并复制k个最近邻的结果
        float* h_distances = new float[k];
        int* h_labels = new int[k];
        cudaMemcpy(h_distances, d_distances, k * sizeof(float), cudaMemcpyDeviceToHost);
        cudaMemcpy(h_labels, d_sort_labels, k * sizeof(int), cudaMemcpyDeviceToHost);

        // Count frequency of labels among nearest neighbors
        // 统计最近邻中各标签的频率
        std::vector<int> labelCount(10, 0);
        for (int i = 0; i < k; ++i) {
            labelCount[h_labels[i]]++;
        }

        // Find most frequent label as prediction
        // 找出出现频率最高的标签作为预测结果
        int predictedLabel = std::distance(labelCount.begin(), 
                                         std::max_element(labelCount.begin(), labelCount.end()));

        // Count correct predictions
        // 统计正确预测
        if (predictedLabel == test_label) {
            correct_predictions++;
        }

        // Output prediction results
        // 输出预测结果
        std::cout << "Test Sample " << t << ": Actual = " << test_label 
                 << ", Predicted = " << predictedLabel << std::endl;

        // Free temporary allocated memory
        // 释放临时分配的内存
        delete[] h_distances;
        delete[] h_labels;
        cudaFree(d_distances);
        cudaFree(d_sort_labels);
        cudaFree(d_test_image);
    }

    // Calculate and output overall accuracy
    // 计算并输出总体准确率
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;
    std::cout << "Accuracy: " << accuracy << "%" << std::endl;

    // Free all allocated memory
    // 释放所有分配的内存
    delete[] h_train_images;
    delete[] h_train_labels;
    cudaFree(d_train_images);
    cudaFree(d_train_labels);

    return 0;
}

In [21]:
%cuda_group_run --group "parseImage" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

Successfully loaded 1000 training samples.
Successfully loaded 10 testing samples.
Test Sample 0: Actual Label = 0, Predicted Label = 0
Test Sample 1: Actual Label = 7, Predicted Label = 7
Test Sample 2: Actual Label = 1, Predicted Label = 1
Test Sample 3: Actual Label = 1, Predicted Label = 1
Test Sample 4: Actual Label = 4, Predicted Label = 4
Test Sample 5: Actual Label = 9, Predicted Label = 9
Test Sample 6: Actual Label = 4, Predicted Label = 4
Test Sample 7: Actual Label = 3, Predicted Label = 3
Test Sample 8: Actual Label = 4, Predicted Label = 4
Test Sample 9: Actual Label = 8, Predicted Label = 8
Accuracy: 100%

